# Advanced Python


### Introduction

In this lecture, we will talk about efficient computation in Python.

*  Python is a vector-oriented language. In most cases, vectorization speeds up computation.
*  We turn to more CPUs for parallel execution to save time if there is no more room to optimize the code to improve the speed.
*  Clusters are accessed remotely. Communicating with a remote cluster is different from operating a local machine.

### Vectorization

Despite mathematical equivalence, various ways of calculation can perform distinctively in terms of computational speed.

Does computational speed matter?
For a job that takes less than a minutes, the time difference is not a big deal.
For modern economic structural estimation problems commonly seen in industrial organization, a single estimation can take up to a week. For those problems code optimization is essential.

Other computational intensive procedures include bootstrap, simulated maximum likelihood and simulated method of moments. Even if a single execution does not take much time, repeating such a procedure for thousands of replications will consume a non-trivial duration.

Of course, optimizing code takes human time. It is a balance of human time and machine time.

__Example__

The heteroskedastic-robust variance for the OLS regression is
$$(X'X)^{-1} X'\hat{e}\hat {e}'X (X'X)^{-1}$$
The difficult part is $X'\hat{e}\hat {e}'X=\sum_{i=1}^n \hat{e}_i^2 x_i x_i'$.
There are at least 4 mathematically equivalent ways to compute this term.

1.  literally sum over $i=1,\dots,n$ one by one.
2.  $X' \mathrm{diag}(\hat{e}^2) X$, with a dense central matrix.
3.  $X' \mathrm{diag}(\hat{e}^2) X$, with a sparse central matrix.
4.  Do cross product to `X*e_hat`. It takes advantage of the element-by-element operation.

We first generate the data of binary response and regressors. Due to the discrete nature of the dependent variable, the error term in the linear probability model is heteroskedastic. It is necessary to use the heteroskedastic-robust variance to consistently estimate the asymptotic variance of the OLS estimator. The code chunk below estimates the coefficients and obtains the residual.

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix 
import random
import datetime
import math
import statistics
import matplotlib.pyplot as plt

c:\Users\zhent\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def lpm(n):
    # estimation in a linear probability model

    # set the parameters
    b0 = np.array([-1, 1])

    # generate the data
    e = np.random.normal(size=n)
    X = np.hstack((np.ones((n, 1)), np.random.normal(size=(n, 1))))
    Y = (X @ b0 + e >= 0)

    # OLS estimation
    bhat = np.linalg.inv(X.T @ X) @ X.T @ Y

    # calculate the t-value
    bhat2 = bhat[1]  # parameter we want to test
    e_hat = Y - X @ bhat
    return X, e_hat


We run the 4 estimators for the same data, and compare the time.

In [4]:
import time
import numpy as np
from scipy.sparse import diags

# an example of robust variance matrix.
# compare the implementation via matrix and vectorization.

n = 5000; Rep = 10 # large matrix

# n = 50; Rep = 1000 # small matrix



for opt in range(1, 5):
    pts0 = time.time()
    
    # initialize the matrix for computing the variance-covariance matrix
    XXe2 = np.zeros((2, 2))

    # loop over the replications and compute the variance-covariance matrix
    for iter in range(Rep):
        np.random.seed(iter)
        data = lpm(n)
        X = data[0]
        e_hat = data[1]
        # compute the variance-covariance matrix using element-by-element calculation
        if opt == 1:
            for i in range(n):
                XXe2 += e_hat[i]**2 * np.matrix(X[i,]).T @ np.matrix(X[i,])
        
        # compute the variance-covariance matrix using matrix multiplication with dense matrices
        elif opt == 2:
            e_hat2_M = np.zeros((n, n))
            np.fill_diagonal(e_hat2_M, e_hat**2)
            XXe2 = np.matrix(X).T @ np.matrix(e_hat2_M) @ np.matrix(X)
        
        # compute the variance-covariance matrix using matrix multiplication with sparse matrices
        elif opt == 3:
            e_hat2_M = diags(e_hat**2, format='csr')
            XXe2 = X.T @ e_hat2_M @ X
        
        # compute the variance-covariance matrix using vectorization with no waste
        elif opt == 4:
            e_hat = e_hat.reshape((-1, 1))
            Xe = np.matrix(X).T * np.matrix(e_hat)
            XXe2 = Xe @ Xe.T
        
        # compute the robust variance-covariance matrix using the sample size
        XX_inv = np.linalg.inv(X.T @ X)
        sig_B = XX_inv @ XXe2 @ XX_inv
        
    print("n =", n, ", Rep =", Rep, ", opt =", opt, ", time =", time.time() - pts0, "\n")


n = 5000 , Rep = 10 , opt = 1 , time = 1.6894004344940186 

n = 5000 , Rep = 10 , opt = 2 , time = 3.044187068939209 

n = 5000 , Rep = 10 , opt = 3 , time = 0.061038970947265625 

n = 5000 , Rep = 10 , opt = 4 , time = 0.021941661834716797 



**Results with n = 50 and Rep = 1000**

n = 50 , Rep = 1000 , opt = 1 , time = 0.5873687267303467 

n = 50 , Rep = 1000 , opt = 2 , time = 0.07624149322509766 

n = 50 , Rep = 1000 , opt = 3 , time = 0.20538115501403809 

n = 50 , Rep = 1000 , opt = 4 , time = 0.06907129287719727 

We clearly see the difference in running time, though the 4 methods are mathematically the same.
When $n$ is small, `matrix` is fast and `Matrix` is slow; the vectorized version is the fastest.
When $n$ is big, `matrix` is slow and `Matrix` is fast; the vectorized version is still the fastest.

## Efficient Loop

* R evolves for big data
* housekeeping is needed in `for` loops
* `plyr` simplifies the job and facilitates parallelization.


### Example

* Empirical coverage probability of a Poisson distribution
* Write a DIY `CI` for confidence interval

This is a standard `for` loop.

In [1]:
import numpy as np
import time

def CI(x):
    # x is a numpy array of random variables
    n = len(x)
    mu = np.mean(x)
    sig = np.std(x)
    upper = mu + 1.96 / np.sqrt(n) * sig
    lower = mu - 1.96 / np.sqrt(n) * sig
    return {"lower": lower, "upper": upper}

This is a standard `for` loop.

In [2]:
Rep = 100 # or whatever value you choose
sample_size = 100 # or whatever value you choose
mu = 10 # or whatever value you choose

np.random.seed(1) # set seed for reproducibility
out = [] # initialize out as empty list
pts0 = time.time() # check time

for i in range(Rep):
    x = np.random.poisson(mu, size=sample_size)
    bounds = CI(x)
    out.append((bounds["lower"] <= mu) & (mu <= bounds["upper"]))

pts1 = time.time() - pts0 # check time elapsed
print("loop without pre-definition takes", pts1, "seconds")

loop without pre-definition takes 0.013939619064331055 seconds


### Classical loop with an empty list

In [3]:
import numpy as np
import time


Rep = 100 # or whatever value you choose
sample_size = 10 # or whatever value you choose
mu = 10 # or whatever value you choose

np.random.seed(1) # set seed for reproducibility
out = np.zeros(Rep, dtype=bool) # pre-define out with appropriate shape
pts0 = time.time() # check time

for i in range(Rep):
    x = np.random.poisson(mu, size=sample_size)
    bounds = CI(x)
    out[i] = ((bounds["lower"] <= mu) & (mu <= bounds["upper"]))

pts1 = time.time() - pts0 # check time elapsed
print("loop with pre-definition takes", pts1, "seconds")

loop with pre-definition takes 0.012025833129882812 seconds


* Pay attention to the line `out = np.empty(Rep, dtype=bool) `. 
* Memoery operates differently with or without the container

### `plyr`
There is no equivalent module in python for `plyr`, so this part is omitted for now.

## Parallel Computing

Parallel computing becomes essential when the data size is beyond the storage of a single computer, for example  {% cite li2018embracing %}.
Here we explore the speed gain of parallel computing on a multicore machine.

Here we explore the speed gain of parallel computing on a multicore machine.

The package `multiprocessing` is the choice for parallel computing in Python.
Below is the basic structure. 

In [4]:
Rep = 100
sample_size = 100
mu = 2

In [5]:
import numpy as np
from scipy.stats import poisson
import time

def capture(i):
    x = np.random.poisson(mu, sample_size)
    bounds = CI(x)  # You need to define the CI function in Python
    return (bounds['lower'] <= mu) & (mu <= bounds['upper'])

pts0 = time.time()  # check time
out = [capture(i) for i in range( Rep)]

pts1 = time.time() - pts0  # check time elapsed
print(pts1)

0.0056362152099609375


* Surprisingly, parallel computing runs more slowly
  * Each loop can be done in very short time.

* code chunk below will tell a different story.
  * Time in each loop is non-trivial.

In [6]:
from multiprocessing import Pool
import time

Rep = 20
sample_size = 2000000

pts0 = time.time()  # check time
out = [capture(i) for i in range(Rep)]  # single-core execution of capture function
pts1 = time.time() - pts0  # check time elapsed
print(f"single-core loop takes {pts1} seconds")


single-core loop takes 3.500565767288208 seconds


In [7]:
import time
from multiprocessing import Pool

def parallel_func(i):
    # your function code here
    # for example:
    x = np.random.poisson(mu, size=sample_size)
    bounds = CI(x)
    return (bounds["lower"] <= mu) & (mu <= bounds["upper"])

Rep = 20 # or whatever value you choose
mu = 10 # or whatever value you choose
sample_size = 2000 # or whatever value you choose

# parallel version with 2 cores
pool = Pool(2) # open 2 CPUs
pts0 = time.time() # check time
out = pool.map(parallel_func, range(Rep))
print("2-core parallel loop takes", time.time() - pts0 , "seconds")


In [13]:

# single-core version
pts0 = time.time()
out = [parallel_func(i) for i in range(Rep)]
print(time.time() - pts0)

single-core loop takes 36.475889682769775 seconds


If we have two CPUs running simultaneously, in theory we can cut the time to a half of that on a single CPU. Is that what happening in practice?

### Multiprocessing with the `process` class

In [22]:
Rep = 10
sample_size = 1000
mu = 2

import multiprocessing as mp
from multiprocessing import Pool
from multiprocessing import Process


def capture(i,return_dict):
    np.random.seed(i)
    x = np.random.poisson(mu, sample_size)
    bounds = CI(x)
    result = ((bounds['lower'] <= mu) & (mu <= bounds['upper']))
    print("The result: " + str(result))
    return_dict[i]=result

pts0 = time.time() # check time

manager = mp.Manager()
return_dict = manager.dict()
jobs = []

for i in range(Rep):
    p = Process(target=capture, args=(i,return_dict))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

print(return_dict.values())

stat_cover = np.mean(return_dict.values()) # jobs.count(True)/Rep*100

print( "empirical coverage probability = ", stat_cover, "% \n") # empirical size
pts1 = time.time() - pts0 # check time elapse
print("The the calculation time is:", pts1, "\n")

The result: True
The result: True
The result: True
The result: True
The result: True
The result: True
The result: True
The result: True
The result: True
The result: True
[True, True, True, True, True, True, True, True, True, True]
empirical coverage probability =  1.0 % 

The the calculation time is: 0.1193692684173584 



### Multiprocessing with the `pool` class & `apply()`

ZT comments: the coverage probability is unlike to be true. Need to check the code.

In [26]:
Rep = 200
sample_size = 2000
mu = 2


pts0 = time.time()  # check time

def capture(i):
    x = np.random.poisson(mu, sample_size)
    bounds = CI(x)
    return ((bounds['lower'] <= mu) & (mu <= bounds['upper']))

# Only allows to run 4 processes at the time
pool = mp.Pool(processes=4)

# Initiate the multiprocess process wit apply()
results = [pool.apply(capture, args=(i,)) for x in range(Rep)]

print( "empirical coverage probability = ", np.mean(results), "\n") # empirical size
pts1 = time.time() - pts0 # check time elapse
print("The the calculation time is:", pts1, "\n")
# print(results) 

empirical coverage probability =  0.955 

The the calculation time is: 0.17428827285766602 



### Multiprocessing with the `pool` class & `map()`

In [28]:
Rep = 200
sample_size = 2000
mu = 2

pts0 = time.time()  # check time
def capture(i):
    x = np.random.poisson(mu, sample_size)
    bounds = CI(x)
    return ((bounds['lower'] <= mu) & (mu <= bounds['upper']))
    
# Only allows to run 4 processes at the time
pool = mp.Pool(processes=4)

# Initiate the multiprocess process with the map()
results = pool.map(capture, range(Rep), )

print( "empirical coverage probability = ", results.count(True)/Rep*100, "\n") # empirical size
pts1 = time.time() - pts0 # check time elapse
print("The the calculation time is:", pts1, "\n")
# print(results) 

empirical coverage probability =  94.5 

The the calculation time is: 0.035188913345336914 

